In [ ]:
import csv
import json
import sys
from pathlib import PosixPath
import pandas as pd

In [ ]:
sequestration_master_file_path = PosixPath(
    "~/CTDS/projects/midrc/indexing-data/sequestration/master_sequestration_locations_23333_2022-06-28.tsv"
).expanduser()

packages_acr = PosixPath("/Users/andrewprokhorenkov/CTDS/projects/midrc/indexing-data/done_packages_acr")
packages_acr = list(packages_acr.glob("**/*.tsv"))

packages_rsna = PosixPath("/Users/andrewprokhorenkov/CTDS/projects/midrc/indexing-data/done_packages_rsna")
packages_rsna = list(packages_rsna.glob("**/*.tsv"))

In [ ]:
seq_master = {}
with open(sequestration_master_file_path) as sequestration_master_file:
    reader = csv.DictReader(sequestration_master_file, delimiter="\t")

    for row in reader:
        seq_master[row["case_ids"]] = row["dataset"]

In [ ]:
files = []

# for package in packages_acr:
for package in packages_rsna:
    case_id = package.parts[-3]
    dataset = seq_master.get(case_id, None)
    
    if dataset == "Open":
        print(case_id)
        data = pd.read_csv(package, sep="\t")

        urls = data["storage_urls"]
        urls = urls.apply(lambda v: v.replace("s3://storage.ir.rsna.ai", ""))
        urls = urls.apply(lambda v: v.replace("//", "replicated-data-acr/"))

        files.extend(urls.to_list())
    # break

In [ ]:
len(files)

In [ ]:
files2 = map(lambda v: v + "\n", files)
with open("list_of_open_acr.txt", "w") as f:
    f.writelines(files2)